In [1]:
!pip install fastbm25

In [1]:
from fastbm25 import fastbm25
import pickle 
import pandas as pd

In [2]:
### load data 

doc = pd.read_json("./data/wiki_data.json",lines=True)
query = pd.read_csv("./data/crawl_context.csv")
index = pd.read_parquet('./wiki_data/my_index.parquet')

In [3]:
a = set([int(x) for x in index['id']])
b = set([int(x) for x in doc['page_id']])
c = set([int(x) for x in query['page_id']])
d = a & b & c

In [4]:
query['check'] = query['page_id'].apply(lambda x : int(x) in d)
doc['check'] = doc['page_id'].apply(lambda x : int(x) in d)
query = query[query['check']].reset_index(drop=True)
doc = doc[doc['check']].reset_index(drop=True)

In [5]:
query

,id,prompt,A,B,C,D,E,answer,wiki_text,page_id,page_title,stem_label,check
0,0,What is physical mathematics?,The study of physically motivated mathematics,The study of mathematical physics,The study of mathematics in physical contexts,The study of mathematical equations,The study of mathematical operations,A,The subject of physical mathematics is concern...,32439784,Physical mathematics,M,True
1,1,Who wrote Physical Arithmetic in 1885?,Margaret Osler,Alexander Macfarlane,Alhazen,Galileo,Newton,B,The subject of physical mathematics is concern...,32439784,Physical mathematics,M,True
2,2,What did the Mathematical Tripos at Cambridge ...,Pure mathematics,Applied mathematics,Mixed mathematics,Fluxional calculus,Physical problems,C,The subject of physical mathematics is concern...,32439784,Physical mathematics,M,True
3,3,What mathematical representation is used for m...,Complex numbers,Quaternions,Linear algebra,Fluxional calculus,Mixed mathematics,C,The subject of physical mathematics is concern...,32439784,Physical mathematics,M,True
4,4,What did the early expressions of kinematics a...,Causality,Forces,Mathematical physics,Fluxional calculus,Mixed mathematics,C,The subject of physical mathematics is concern...,32439784,Physical mathematics,M,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45066,49279,What is the purpose of acoustic cleaning?,To remove material from surfaces,To generate powerful sound waves,To handle bulk granular materials,To reduce the need for manual cleaning,To build material-handling equipment,A,Acoustic cleaning is a maintenance method used...,4117074,Acoustic cleaning,E,True
45067,49280,How does an acoustic cleaner work?,By generating powerful sound waves,By shaking particulates loose from surfaces,By removing the buildup of material on surfaces,By reducing the need for manual cleaning,By handling bulk granular materials,B,Acoustic cleaning is a maintenance method used...,4117074,Acoustic cleaning,E,True
45068,49281,What is an acoustic cleaner made of?,Compressed air,Electricity,A diaphragm,Stainless steel,Titanium,D,Acoustic cleaning is a maintenance method used...,4117074,Acoustic cleaning,E,True
45069,49282,Why is compressed air used to power an acousti...,To generate powerful sound waves,To shake particulates loose from surfaces,To ensure performance and longevity,To reduce the need for manual cleaning,To avoid sparking and explosions,E,Acoustic cleaning is a maintenance method used...,4117074,Acoustic cleaning,E,True


In [6]:
doc

,page_id,title,content,check
0,32439784,Physical mathematics,The subject of physical mathematics is concern...,True
1,32439784,Physical mathematics,The subject of physical mathematics is concern...,True
2,32439784,Physical mathematics,The subject of physical mathematics is concern...,True
3,32439784,Physical mathematics,The subject of physical mathematics is concern...,True
4,32439784,Physical mathematics,The subject of physical mathematics is concern...,True
...,...,...,...,...
45066,59006692,Trajectory inference,Trajectory inference or pseudotemporal orderin...,True
45067,11356477,List of mammals of Turkmenistan,This is a list of the mammal species recorded ...,True
45068,11356477,List of mammals of Turkmenistan,This is a list of the mammal species recorded ...,True
45069,799405,Shear mapping,"In plane geometry, a shear mapping is a linear...",True


In [7]:
doc_dup  = doc.drop_duplicates(['content'])
doc_dup.shape

(8662, 4)

In [8]:
doc_dup['text'] = doc_dup['title']+" "+doc_dup['content']

/tmp/ipykernel_1431774/3280834534.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doc_dup['text'] = doc_dup['title']+" "+doc_dup['content']


In [9]:
doc_dup.head()

,page_id,title,content,check,text
0,32439784,Physical mathematics,The subject of physical mathematics is concern...,True,Physical mathematics The subject of physical m...
5,38183293,DaVinci (software),DaVinci was a development tool used to create ...,True,DaVinci (software) DaVinci was a development t...
10,1997,Algebraic geometry,Algebraic geometry is a branch of mathematics ...,True,Algebraic geometry Algebraic geometry is a bra...
15,10952377,Material-handling equipment,Material handling equipment (MHE) is mechanica...,True,Material-handling equipment Material handling ...
20,1219747,Concrete mixer,A concrete mixer (also cement mixer) is a devi...,True,Concrete mixer A concrete mixer (also cement m...


In [10]:
doc_dict = dict(zip(doc_dup['text'].values,doc_dup['page_id'].values))

In [11]:
####bm25
from tqdm.auto import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np
# nltk.download('punkt')
# nltk.download('stopwords')

def word_process(text):
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.isalnum()]
    return words
    

corpus = list(doc_dup['text'])
tokenized_corpus = [word_process(doc) for doc in corpus]
print(np.sum([len(x) for x in tokenized_corpus]))
bm25 = fastbm25(tokenized_corpus)
# with open("bm25_model.pkl","wb")as f:
#     pickle.dump(bm25,f)

/root/conda_env/chr_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


11438336


TypeError: __init__() got an unexpected keyword argument 'num_threads'

In [15]:
len(tokenized_corpus)

8662

In [17]:
#### load query
val = pd.read_csv("./data/recall_val.csv")
val['check'] = val['page_id'].apply(lambda x : int(x) in d)
val = val[val['check']].reset_index(drop=True)

In [18]:
val.head()

,id,prompt,A,B,C,D,E,answer,wiki_text,page_id,page_title,stem_label,prompt_answer,check
0,47000,What are some adverse effects associated with ...,Burns and eczema,Cancer and brain disorder,Fetotoxicity and inflammations of peripheral n...,All of the above,None of the above,D,A solvent is a substance that is liquid at the...,73305281,Green solvent,M,What are some adverse effects associated with ...,True
1,47001,What is the goal of the UN sustainability deve...,To promote the use of petrochemical solvents,To develop green and sustainable solvents,To increase the use of nonpolar solvents,To classify solvents into polar and nonpolar c...,To decrease the use of solvents in various dom...,B,A solvent is a substance that is liquid at the...,73305281,Green solvent,M,What is the goal of the UN sustainability deve...,True
2,47002,What are green solvents derived from?,Processing of agricultural crops,Petrochemical sources,Regular solvents,Chemical reactions,Water and air pollution,A,A solvent is a substance that is liquid at the...,73305281,Green solvent,M,What are green solvents derived from? Processi...,True
3,47003,What makes it difficult to classify green solv...,Their chemical structure and source can differ,They have adverse effects on health and safety,They are associated with water and air pollution,They are derived from petrochemical sources,They are nonpolar solvents,A,A solvent is a substance that is liquid at the...,73305281,Green solvent,M,What makes it difficult to classify green solv...,True
4,47004,What is the significance of the Nora Stone?,It is the oldest Phoenician inscription found ...,It is the only Phoenician inscription found in...,It is the first evidence of Phoenician coloniz...,It is the longest Phoenician inscription ever ...,It is the only known inscription honoring a go...,A,The Nora Stone or Nora Inscription is an ancie...,23809512,Nora Stone,T,What is the significance of the Nora Stone? It...,True


In [19]:
val['prompt_answer'].values[0]

'What are some adverse effects associated with traditional organic solvents? Burns and eczema Cancer and brain disorder Fetotoxicity and inflammations of peripheral nerves All of the above None of the above'

In [20]:
stop_words = set(stopwords.words('english'))


def query_process(q):
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.isalnum()]
    keywords = [word for word in words if word not in stop_words]
    return " ".join(keywords)

def recall_doc(q,corpus):
    q = word_process(q)
    res_list = [corpus[item[1]] for item in bm25.top_k_sentence(q,  k=10)]
    # print(res_list)
    res_ids = []
    for res in res_list:
        res_ids.append(doc_dict[res])
        
    return res_ids
    

In [21]:
recall_doc(val['prompt_answer'].values[0],corpus)

[73305281,
 28695746,
 27024757,
 649100,
 1851370,
 189371,
 35954442,
 471852,
 2574377,
 674]

In [22]:
val["recall"] = val['prompt_answer'].map(lambda x:recall_doc(x,corpus))

In [23]:
val['top1_flag'] = val.apply(lambda x:1 if x['recall'][0] == x['page_id'] else 0,1)


In [24]:
val['top1_flag'].sum() / len(val)

0.9286049788633162

In [25]:
val['top10_flag'] = val.apply(lambda x:1 if  x['page_id'] in x['recall'] else 0,1)


In [26]:
val['top10_flag'].sum() / len(val)

0.9873179896665101